## <span style='color:#ff5f27'> 📝 Imports

In [1]:
import requests
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import re

## <span style='color:#ff5f27'> 🧑🏻‍🏫 Functions

In [2]:
def parse_data(parser,tag: str, class_: str):
    new_data = []
    data = parser.find_all(tag,class_)
    for el in data:
        new_data.append(el.text)
    
    return new_data

In [3]:
def parse_page(page_name):
    response = requests.get(f'https://www.elbruk.se/timpriser-{page_name}')
    content = response.content
    
    parser = BeautifulSoup(content, 'html.parser')
    
    data = parse_data(parser,'div','info-box-content')
    electricity_prices = [float(re.findall(r'\d+\,\d+',info.split('\n')[2])[0].replace(',','.')) for info in data[:4]]
    
    return electricity_prices

## <span style='color:#ff5f27'> 🕵🏻‍♂️ Parsing Electricity Data

In [4]:
page_names = ['se1-lulea','se2-sundsvall','se3-stockholm','se4-malmo']

data = [[datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),page_name.split('-')[1].capitalize(),*parse_page(page_name)] for page_name in page_names]

data

[['2022-09-02 20:00', 'Lulea', 45.72, 42.02, 39.05, 65.69],
 ['2022-09-02 20:00', 'Sundsvall', 45.72, 42.02, 39.05, 65.69],
 ['2022-09-02 20:00', 'Stockholm', 321.75, 483.3, 39.05, 625.09],
 ['2022-09-02 20:00', 'Malmo', 451.35, 602.09, 203.32, 625.09]]

## <span style='color:#ff5f27'> 👩🏻‍🔬 DataFrame Creation/Update

In [5]:
columns = [
    'date',
    'city',
    'price_day',
    'price_current',
    'price_min',
    'price_max'
]

In [6]:
dataframe = pd.DataFrame(
    data = data,
    columns = columns
)

dataframe.head()

,date,city,price_day,price_current,price_min,price_max
0,2022-09-02 20:00,Lulea,45.72,42.02,39.05,65.69
1,2022-09-02 20:00,Sundsvall,45.72,42.02,39.05,65.69
2,2022-09-02 20:00,Stockholm,321.75,483.30,39.05,625.09
3,2022-09-02 20:00,Malmo,451.35,602.09,203.32,625.09


In [7]:
try:
    df_historical = pd.read_csv('electricity_prices.csv')
    df_historical = pd.concat([df_historical,dataframe],ignore_index=True,)
    df_historical.to_csv('electricity_prices.csv',index=False)
except:
    dataframe.to_csv('electricity_prices.csv',index=False)

## <span style='color:#ff5f27'> 👨🏻‍⚖️ Electricity Prices Data Check

In [8]:
df = pd.read_csv('electricity_prices.csv')

df.head()

,date,city,price_day,price_current,price_min,price_max
0,2022-09-02 19:43,Lulea,45.72,40.40,39.05,65.69
1,2022-09-02 19:43,Sundsvall,45.72,40.40,39.05,65.69
2,2022-09-02 19:43,Stockholm,321.75,532.13,39.05,625.09
3,2022-09-02 19:43,Malmo,451.35,590.86,203.32,625.09
4,2022-09-02 19:46,Lulea,45.72,40.40,39.05,65.69


In [9]:
df.shape

(20, 6)

---